In [ ]:
# install pip dependencies
! pip install kagglehub
! pip install pandas
! pip install torch datasets
! pip install ipdb
! pip install seaborn

In [ ]:
# kaggle authentication
# upload kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# import libraries
import kagglehub
import os
import pandas as pd
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import gc
import numpy as np
from tqdm import tqdm
import io
import tarfile
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
# download data to paraquet
download_path = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")
raw_csv_files = glob.glob(os.path.join(download_path, "*.csv"))
print(raw_csv_files)

In [ ]:
# split csvs into managable chunks

CSV_FILES = []

num_raw_files = len(raw_csv_files)
for i, file in enumerate(raw_csv_files):
  print(f"splitting: {i+1}/{num_raw_files}")
  filename = os.path.splitext(os.path.basename(file))[0]
  max_bytes = 500 * 1024 * 1024

  part_idx = 1
  current_size = 0

  with open(file, 'r') as infile:
    header = infile.readline()
    lines_buffer = []
    for line in infile:
      lines_buffer.append(line)
      current_size += len(line.encode('utf-8'))  # size in bytes

      # If current chunk exceeds the limit, write to file
      if current_size >= max_bytes:
        out_path = os.path.join(f"{filename}_part{part_idx}.csv")
        with open(out_path, 'w') as outfile:
          outfile.write(header)
          outfile.writelines(lines_buffer)
        CSV_FILES.append(out_path)
        part_idx += 1
        lines_buffer = []
        current_size = 0

    # Write remaining lines
    if lines_buffer:
      out_path = os.path.join(f"{filename}_part{part_idx}.csv")
      with open(out_path, 'w') as outfile:
        outfile.write(header)
        outfile.writelines(lines_buffer)
      CSV_FILES.append(out_path)

In [ ]:
CSV_FILES = glob.glob("*.csv")
print(CSV_FILES)
sorted_csv = sorted(CSV_FILES)

In [ ]:
test_df = pd.read_csv(CSV_FILES[1])
NUM_COLUMNS = len(test_df.columns)
print(test_df.columns)
del test_df
gc.collect()

In [ ]:

runs = {}
curr_run = ""
curr_len = 0

for file in sorted_csv:
    df = pd.read_csv(file)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed', errors="coerce")
    df = df.sort_values(by='Timestamp')

    for l, t in zip(df['Label'], df['Timestamp']):
        if pd.isna(t):
            continue
        if l == curr_run:
            curr_len += 1
        else:
            if curr_run != "":
                # Update count of this run length for this label
                if curr_run not in runs:
                    runs[curr_run] = {}

                if curr_len not in runs[curr_run]:
                    runs[curr_run][curr_len] = 1
                else:
                    runs[curr_run][curr_len] += 1

            # Start new run
            curr_len = 1
            curr_run = l

    # Final run in this file
    if curr_run != "":
        if curr_run not in runs:
            runs[curr_run] = {}
        if curr_len not in runs[curr_run]:
            runs[curr_run][curr_len] = 1
        else:
            runs[curr_run][curr_len] += 1

    curr_len = 0
    curr_run = ""
print(runs)


In [ ]:
data = []
for label, lengths in runs.items():
    for run_length, count in lengths.items():
        data.append({'Label': label, 'Run Length': run_length, 'Frequency': count})

df_runs = pd.DataFrame(data)
df_runs = df_runs[df_runs['Run Length'] <= 30]

pivot_df = df_runs.pivot(index='Run Length', columns='Label', values='Frequency').fillna(0)

# Plotting the stacked bar graph
pivot_df.plot(kind='bar', stacked=True, figsize=(10, 6))

plt.title('Frequency of Run Lengths by Label')
plt.xlabel('Run Length')
plt.ylabel('Frequency')
plt.ylim(top=600000)
plt.xticks(rotation=45)
plt.legend(title='Label')
plt.tight_layout()
plt.show()

In [ ]:
window_size = 10
counts_per_window = []

for file in tqdm(sorted_csv):
    df = pd.read_csv(file)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed', errors="coerce")
    df = df.sort_values(by='Timestamp')

    for i in range(len(df) - window_size + 1):
        window = df['Label'].iloc[i:i+window_size]
        count = Counter(window)
        counts_per_window.append(count)

In [ ]:
counts_df = pd.DataFrame(counts_per_window).fillna(0).astype(int)
counts_df['Malicious'] = counts_df.drop(columns='Benign').sum(axis=1)
# counts_df = counts_df[['Benign', 'Malicious']]
#print(counts_df)

In [ ]:
counts_df = counts_df[counts_df['Malicious'] > 0]
print(counts_df['Malicious'].value_counts())
#counts_df.plot(kind='bar', stacked=True)

frequencies = counts_df['Malicious'].value_counts()

frequencies.plot(kind='bar')
plt.title('Frequency of Appearences in 10 wide frame')
plt.xlabel('Appearences')
plt.ylabel('Frequency')
#plt.ylim(top=600000)
plt.xticks(rotation=45)
#plt.legend(title='Label')
plt.tight_layout()
plt.show()

In [ ]:
# canon columns
CANON_COLUMN_INDEX = ['Fwd IAT Tot', 'Fwd Pkt Len Min', 'Down/Up Ratio', 'Dst Port', 'Fwd IAT Std', 'Fwd Header Len', 'Fwd IAT Min', 'Flow IAT Std', 'Active Std', 'Bwd IAT Max', 'Fwd Pkt Len Mean', 'Pkt Size Avg', 'PSH Flag Cnt', 'Flow IAT Mean', 'Fwd Act Data Pkts', 'Bwd Pkt Len Max', 'Flow IAT Max', 'ACK Flag Cnt', 'Bwd IAT Tot', 'Flow IAT Min', 'Bwd Pkts/b Avg', 'Fwd IAT Max', 'SYN Flag Cnt', 'Bwd Header Len', 'Fwd Seg Size Avg', 'Bwd Byts/b Avg', 'Subflow Bwd Byts', 'Pkt Len Max', 'Bwd Pkts/s', 'Fwd IAT Mean', 'Pkt Len Var', 'Fwd Pkt Len Std', 'Protocol', 'Init Bwd Win Byts', 'Active Min', 'Src Port', 'RST Flag Cnt', 'Subflow Fwd Byts', 'Init Fwd Win Byts', 'Bwd Pkt Len Std', 'Fwd PSH Flags', 'Fwd Pkts/s', 'Bwd Blk Rate Avg', 'Flow Byts/s', 'CWE Flag Count', 'Pkt Len Std', 'Active Max', 'Fwd Byts/b Avg', 'Fwd Blk Rate Avg', 'URG Flag Cnt', 'Timestamp', 'Fwd Pkts/b Avg', 'Idle Mean', 'Idle Std', 'Fwd Pkt Len Max', 'Pkt Len Min', 'Flow Duration', 'Fwd Seg Size Min', 'Bwd IAT Min', 'TotLen Fwd Pkts', 'Flow Pkts/s', 'Active Mean', 'ECE Flag Cnt', 'Idle Min', 'Subflow Bwd Pkts', 'Bwd Pkt Len Mean', 'Pkt Len Mean', 'Tot Fwd Pkts', 'Bwd IAT Std', 'Bwd Seg Size Avg', 'Bwd URG Flags', 'Bwd Pkt Len Min', 'Tot Bwd Pkts', 'Subflow Fwd Pkts', 'Bwd IAT Mean', 'FIN Flag Cnt', 'Bwd PSH Flags', 'TotLen Bwd Pkts', 'Fwd URG Flags', 'Idle Max']
CANON_COLUMN_INDEX.sort()
CANON_COLUMN_INDEX.append('Label')
print(CANON_COLUMN_INDEX)
TRAINING_UNWANTED_COLUMNS = ['Timestamp', 'Flow ID', 'Dst IP', "Src IP"]

In [ ]:
# get normalization stats
UNWANTED_COLUMNS_STATS = ['Timestamp','Label', 'Flow ID', 'Dst IP', "Src IP"]

def calcStatistics(files):
  n_total = 0
  mean_total = pd.Series(0.0)
  M2 = pd.Series(0.0)

  file_idx = 0
  num_files = len(files)
  for file in files:
    print(f"calculating: {file_idx+1}/{num_files}")
    df = pd.read_csv(file)
    df = df.reindex(columns=CANON_COLUMN_INDEX)
    for col in df.columns:
      if col not in UNWANTED_COLUMNS_STATS:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')

        if col not in mean_total:
          mean_total[col] = 0.0
          M2[col] = 0.0

      else:
        df = df.drop(columns=[col])

    n = len(df)
    mean = df.mean()
    var = df.var(ddof=0)

    # Welford’s algorithm for combining means and variances
    delta = mean - mean_total
    new_n = n_total + n

    mean_total = (n_total * mean_total + n * mean) / new_n
    M2 += var * n + (delta**2) * n_total * n / new_n

    n_total = new_n

    file_idx += 1
    del df
    gc.collect()

  std = (M2 / n_total) ** 0.5
  length = n_total
  mean = mean_total
  return (length, mean, std)

LENGTH, MEAN, STD = calcStatistics(CSV_FILES)

import json
j = {'length': LENGTH, 'mean': MEAN.to_dict(), 'std': STD.to_dict(), 'non-stat-columns': UNWANTED_COLUMNS_STATS}
with open('info.json', 'w') as f:
  json.dump(j,f)

In [ ]:
# normalize dataset and convert to parquet
! mkdir "normalized-benign/"

UNWANTED_COLUMNS_NORM = ['Flow ID', 'Dst IP', "Src IP"]

def normalize(files, mean, std):
  pq_files = []

  num_files = len(files)
  for i,file in enumerate(CSV_FILES):
    print(f"normalizing: {i+1}/{num_files}")
    df = pd.read_csv(file)
    df = df.reindex(columns=CANON_COLUMN_INDEX)
    filename = os.path.splitext(os.path.basename(file))[0]
    name = f'normalized-benign/normalized_{filename}-benign.parquet'
    pq_files.append(name)

    for col in df.columns:
      if col not in UNWANTED_COLUMNS_STATS:
          df[col] = pd.to_numeric(df[col], errors='coerce').astype('float32')
          df[col] = (df[col] - mean[col]) / std[col]
      else:
        if col in UNWANTED_COLUMNS_NORM:
          df = df.drop(columns=[col])

    if 'Src Port' not in df.columns:
      df['Src Port'] = np.nan

    df = df[df['Label'] == "Benign"]

    df.to_parquet(name)

    del df
    gc.collect()
  return pq_files

PARQUET_FILES = normalize(CSV_FILES, MEAN, STD)
print(PARQUET_FILES)

In [ ]:
# double check columns
base_df = pd.read_parquet(PARQUET_FILES[0])
base = set(list(base_df.columns))
del base_df
gc.collect()

for f in PARQUET_FILES:
  df = pd.read_parquet(f)

  print(df[df['Label'] != "Benign"])

  cols = set(list(df.columns))
  if base != cols:
    print(f'in base: {base - cols}, in cols: {cols - base}')
  del df
  gc.collect()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
! tar -czvf normalized-benign-ids2018-parquet.tar.gz normalized-benign/*
# ! cp normalized-ids2018.tar.gz drive/MyDrive/